# Computational Finance Assignment1

In [ ]:
import math
import sympy as sp
import numpy as np

## Part I Theory: derivatives and no-arbitrage

### 1. Prove:

`A = C(1 + r/n)^(nt) = C * e^(rt)`

The standard compound interest formula is: 

`A = C(1 + r/n)^(nt)`, where:

- `A` is the future value,
- `C` is the principal (initial investment),
- `r` is the annual interest rate,
- `n` is the number of compound interest per year,
- `t` is time in years.

As we let `n` go to infinity, we can use the property of the base `e` of the natural logarithm, `(1 + 1/n)^n` tends to `e` as `n` goes to infinity. We set `n = 1,10,100,1000,10000` to compare these two results


In [ ]:
C = 1 
r = 0.1
t = 1

A_true = C * math.exp(r * t) 
ns = [1,10,100,1000,10000]
for n in ns:
    A_n= C * pow(1 + r/n, n * t)
    absolute_error = abs(A_true - A_n)/A_true
    print(f"Continuous compound interest : {A_true} ", f"Standard compound interest (with n = {n}): {A_n}")
    print(f"Absolute Error:{absolute_error}")
    print("\n")


### 2.  Calculate the fair-value of a coupon bond.

In [ ]:


# Calculate the fair value of a bond
# Given parameters
face_value = 50000  # Face value of the bond
coupon = 300  # Coupon payment per quarter
maturity = 2  # Maturity period in years
risk_free_rate = 0.015  # Risk-free interest rate
quarters = maturity * 4  # Total number of quarters for payments

# Calculate the present value of each quarter
present_value_coupons = sum(coupon * sp.exp(-risk_free_rate * t/4) for t in range(1, quarters + 1))

# Present value of the final principal payment
present_value_face_value = face_value * sp.exp(-risk_free_rate * maturity)

# The fair value of the bond is the sum of present values of all cash flows
fair_value_bond = present_value_coupons + present_value_face_value

print(f"Fair-value of this coupon bond : {fair_value_bond.evalf()}")  # Evaluate the numerical result of the expression




### 3. 

### 4. 

### 5. Prove put-call parity：`C_t + Ke^(-r(T-t)) = P_t + S_0`
#### Portfolio A:
- Consists of one European call option with the value `C_t` and an investment of `Ke^(-r(T-t))` in the risk-free asset. The payoff at maturity is the greater of `S_T - K` or 0, plus `K`, which simplifies to `max(S_T, K)`.

#### Portfolio B:
- Consists of one European put option with the value `P_t` and one share of the stock `S_0`. The payoff at maturity is the greater of `K - S_T` or 0, plus `S_T`, which simplifies to `max(S_T, K)`.

Given that the payoff of both portfolios is the same at maturity, and no arbitrage opportunities exist, their value should be equal at the present time. This leads to the equation:

`C_t + Ke^(-r(T-t)) = P_t + S_0`


## Part II: Binomial tree: option valuation

#### 1. Build the binomial rree and test

In [ ]:
def buildTree(S, vol, T, N):
    '''
    Build a binomial tree for option pricing.
    Paramers:
        S: Initial stock price
        vol: Volatility
        T: Time to expiration
        N: Number of steps
    Output:
        A 2D array representing the binomial tree
    '''
    dt = T / N  # Time step
    u = np.exp(vol * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    matrix = np.zeros((N + 1, N + 1))  # Create a 2D array for tree to store the stock prices
    
    for i in range(N + 1): 
        for j in range(i + 1):  
            matrix[i, j] = S * (u ** j) * (d ** (i - j))
            
    return matrix

def valueOptionMatrix(tree, T, r, K, vol):
    '''
    Calculate the option value using a binomial tree.
    Parameters:
        tree: Binomial tree
        T: Time to expiration
        r: Risk-free interest rate
        K: Strike price
        vol: Volatility
    Output:
        Option value at the root of the tree
    '''
    N = tree.shape[0] - 1  # Number of steps
    dt = T / N 
    u = np.exp(vol * np.sqrt(dt)) 
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    for c in range(N + 1):
        tree[N, c] = max(0, tree[N, c] - K) # Option value at expiration
    
    for i in range(N - 1, -1, -1): 
        for j in range(i + 1):
            # Calculate the expected value of the option
            tree[i, j] = np.exp(-r * dt) * (p * tree[i + 1, j + 1] + (1 - p) * tree[i + 1, j])
            
    return tree[0, 0]

In [ ]:
sigma = 0.2  # Volatility
S = 100  # Current stock price
T = 1.0  # Time to maturity
N = 50  # Number of steps
K = 99  # Strike price
r = 0.06  # Risk-free interest rate

tree = buildTree(S, sigma, T, N)
option_price = valueOptionMatrix(tree, T, r, K, sigma)
option_price

#### 2. 